# **IMDb Top 400 Movies Analysis** 

First, we select the top 5 columns of the IMDB Top 400 table to see what the table looks like.

In [51]:
SELECT TOP 5 * FROM MoviesDB.dbo.IMDBtop400;

(5 rows affected)

Total execution time: 00:00:00.010

Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info,Country_of_Origin,Language
1. The Shawshank Redemption (1994),R,142 min,Drama,9.3,80,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.","Director: Frank Darabont | Stars: Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler","Votes: 2,295,987 | Gross: $28.34M",USA,English
2. The Godfather (1972),R,175 min,"Crime, Drama",9.2,100,The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.,"Director: Francis Ford Coppola | Stars: Marlon Brando, Al Pacino, James Caan, Diane Keaton","Votes: 1,584,782 | Gross: $134.97M",USA,English
3. The Dark Knight (2008),PG-13,152 min,"Action, Crime, Drama",9,84,"When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.","Director: Christopher Nolan | Stars: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine","Votes: 2,260,649 | Gross: $534.86M",USA,English
4. The Godfather: Part II (1974),R,202 min,"Crime, Drama",9,90,"The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate.","Director: Francis Ford Coppola | Stars: Al Pacino, Robert De Niro, Robert Duvall, Diane Keaton","Votes: 1,107,253 | Gross: $57.30M",USA,English
5. The Lord of the Rings: The Return of the King (2003),PG-13,201 min,"Action, Adventure, Drama",8.9,94,Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring.,"Director: Peter Jackson | Stars: Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom","Votes: 1,614,369 | Gross: $377.85M",USA,English


Here's a description of the various columns in the table:

1. **Title**/ranking: the title and ranking of the film;
    
2. **Certificate**: MPAA rating (NC-17: no one under 17 admitted, R: restricted, PG: parental guidance, PG-13, Approved: appropriate for all audiences). Many earlier films are also unrated; 
    
3. **Duration**: length of the film;
    
4. **Genre**: the genre(s) of the film (most films have multiple dramas); 
    
5. **Rate**: IMDb rating (out of 10), the average of all the IMDb user ratings; 
    
6. **Metascore**: Metacritic score (out of 100), a weighted score based on popularity, stature, and volume of reviews; 
    
7. **Cast**: director and actors of the film (original dataset does not separate the two, so we will have to); 
    
8. **Info**: the total number of votes submitted by IMDb users, and box office gross in USD; 
    
9. **Country\_of\_Origin**: I added this column after extensive googling; main country of production; some movies were a coproduction between multiple countries. While some movies (like _Das Boot_) were produced in West Germany, I decided to consider West Germany as Germany to consolidate my data. However, I decided to keep the now extinct country the Soviet Union because the country had so many member countries that I had no information about which member country produced the movie.  I also considered Hong Kong and China to be two different countries (and British Hong Kong and the UK to be different as well). 
    
10. **Language**: language(s) of the original release of the movie.

# **Data Cleanup**

First, we need to clean the data to make processing it easier. The first thing I noticed was the data was missing two rows in the 199 and 200th spots, which I replaced.

In [82]:
--REPLACE MISSING ROWS 
INSERT INTO MoviesDB.dbo.IMDBtop400 (Title, Certificate, Duration, Genre, Rate, Metascore, Description, Cast, Info, Country_of_Origin, Language) 
VALUES (N'199. Z (1969)', N'PG', N'127 min', N'Crime,Drama,Thriller', 8.2, 86, N'The public murder of a prominent politician and doctor amid a violent demonstration is covered up by military and government officials. A tenacious magistrate is determined not to let them get away with it.',
N'Director: Costa-Gavras | Stars: Yves Montand,Irene Papas,Jean-Louis Trintignant,François Perier', N'Votes: 29,372 | Gross: $0.08M', N'Algeria', N'French'),
(N'200. Dersu Uzala (1975)', N'Approved', N'142 min', N'Adventure, Biography, Drama', 8.2, NULL, 
N'The Russian army sends an explorer on an expedition to the snowy Siberian wilderness where he makes friends with a seasoned local hunter.',
N'Director: Akira Kurosawa | Stars: Maksim Munzuk, Yuriy Solomin, Mikhail Bychkov, Vladimir Khrulyov', 'Votes: 30,641 | Gross: $1.2M', N'Soviet Union, Japan', N'Russian');

(2 rows affected)

Total execution time: 00:00:00.016

I also decided to get rid of the spaces between commas, which will make analysis easier later when I apply substring searches.

In [84]:
UPDATE MoviesDB.dbo.IMDBtop400
SET Genre = REPLACE(Genre, ', ', ','), 
Country_of_Origin = REPLACE(Country_of_Origin, ', ', ','),
Language = REPLACE(Language, ', ', ','),
Cast = REPLACE(Cast, ', ', ',');

(400 rows affected)

Total execution time: 00:00:00.046

In [93]:
UPDATE MoviesDB.dbo.IMDBtop400
SET Country_of_Origin = REPLACE(Country_of_Origin, 'Portuguese', 'Brazil'),
Language = REPLACE(Language, 'Brazil', 'Portuguese')
WHERE Title = '344. Elite Squad 2: The Enemy Within (2010)'; 

(1 row affected)

Total execution time: 00:00:00.010

Displaying the table will show that the spaces between commas are indeed now gone.

In [86]:
SELECT TOP 5 * FROM MoviesDB.dbo.IMDBtop400; 

(5 rows affected)

Total execution time: 00:00:00.010

Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info,Country_of_Origin,Language
1. The Shawshank Redemption (1994),R,142 min,Drama,9.3,80,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.","Director: Frank Darabont | Stars: Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler","Votes: 2,295,987 | Gross: $28.34M",USA,English
2. The Godfather (1972),R,175 min,"Crime,Drama",9.2,100,The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.,"Director: Francis Ford Coppola | Stars: Marlon Brando,Al Pacino,James Caan,Diane Keaton","Votes: 1,584,782 | Gross: $134.97M",USA,English
3. The Dark Knight (2008),PG-13,152 min,"Action,Crime,Drama",9,84,"When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.","Director: Christopher Nolan | Stars: Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine","Votes: 2,260,649 | Gross: $534.86M",USA,English
4. The Godfather: Part II (1974),R,202 min,"Crime,Drama",9,90,"The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate.","Director: Francis Ford Coppola | Stars: Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton","Votes: 1,107,253 | Gross: $57.30M",USA,English
5. The Lord of the Rings: The Return of the King (2003),PG-13,201 min,"Action,Adventure,Drama",8.9,94,Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring.,"Director: Peter Jackson | Stars: Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom","Votes: 1,614,369 | Gross: $377.85M",USA,English


# **Data Analysis**

We are now ready to begin analyzing the data. A natural place to start is analysis of IMDb ratings. 

**IMDb ratings analysis:** First, we can look at the breakdown of the ratings. A preliminary analysis showed that most ratings were between 8.0 and 8.4, so I decided to increment the ratings by 0.5 and split the 8.0 - 8.4 tier in two. The results show that a plurality of ratings are either 8.0 or 8.1, with 9.0 and above being extremely rare and only reserved for the most exceptional and popular films and none of the ratings below 8.0.

In [89]:
-- Breakdown of Imdb ratings 
(SELECT 
'> 8.9' AS 'IMDb Rating', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Rate > 8.9)
UNION ALL
(SELECT 
'8.5 - 8.9' AS 'IMDb Rating', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Rate > 8.4 AND Rate < 9.0)
UNION ALL
(SELECT 
'8.2 - 8.4' AS 'IMDb Rating', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Rate > 8.1 AND Rate < 8.5)
UNION ALL
(SELECT 
'8.0 - 8.1' AS 'IMDb Rating', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Rate > 7.9 AND Rate < 8.2)
UNION ALL
(SELECT '< 8.0' AS 'IMDb Rating', COUNT(title) AS 'Count'
FROM MoviesDB.dbo.IMDBtop400
WHERE Rate < 8.0);

(5 rows affected)

Total execution time: 00:00:00.005

IMDb Rating,Count
> 8.9,4
8.5 - 8.9,49
8.2 - 8.4,140
8.0 - 8.1,207
< 8.0,0


**Metascore analysis:** While IMDb ratings were concentrated in one range (8.0 - 8.5), Metascores are all over the place, although all are at least 60, and about a quarter of the movies do not have a metascore.

In [90]:
-- Metascore breakdown 
(SELECT 
'> 95' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore > 94)
UNION ALL
(SELECT 
'90 - 94' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore < 95 AND Metascore > 89)
UNION ALL
(SELECT 
'85 - 89' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore > 84 AND Metascore < 90)
UNION ALL
(SELECT 
'80 - 84' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore > 79 AND Metascore < 85)
UNION ALL
(SELECT 
'75 - 79' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore > 74 AND Metascore < 80)
UNION ALL
(SELECT 
'70 - 74' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore > 69 AND Metascore < 75)
UNION ALL
(SELECT 
'65 - 69' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore > 64 AND Metascore < 70)
UNION ALL
(SELECT 
'60 - 64' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore > 59 AND Metascore < 65)
UNION ALL
(SELECT 
'< 60' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore < 60)
UNION ALL
(SELECT 
'None' AS 'Metascore', COUNT(title) AS 'Count' 
FROM MoviesDB.dbo.IMDBtop400
WHERE Metascore IS NULL);


(10 rows affected)

Total execution time: 00:00:00.006

Metascore,Count
> 95,38
90 - 94,42
85 - 89,53
80 - 84,39
75 - 79,43
70 - 74,32
65 - 69,28
60 - 64,12
< 60,0
None,113


**Country of origin analysis:** Another piece of information we can look for is the breakdown by country of origin. Unsurprisingly, over half of the movies were produced in the United States, where Hollywood is located. India and Japan also dominate the list, as Bollywood and Japanese animated features also have a strong international following.

In [94]:
-- Analysis of countries 
SELECT value as 'Country', COUNT(*) AS 'Count'
FROM MoviesDB.dbo.IMDBtop400  
    CROSS APPLY STRING_SPLIT(Country_of_Origin, ',')  
GROUP BY value    
ORDER BY COUNT(*) DESC;

(35 rows affected)

Total execution time: 00:00:00.006

Country,Count
USA,229
India,48
Japan,27
France,17
UK,12
South Korea,9
Germany,9
Sweden,7
Turkey,7
Soviet Union,6


**Language analysis:** Similarly, we can analyze what languages are most commonly spoken. Unsurprisingly, once again, English tops the list, with Hindi, a major language spoken in Bollywood movies, coming in second, and Japanese coming in third. Note that 4 Indian movies were not filmed in Hindi.

In [95]:
-- Languages breakdown  
SELECT value as 'Language', COUNT(*) AS 'Count'
FROM MoviesDB.dbo.IMDBtop400  
    CROSS APPLY STRING_SPLIT(Language, ',')  
GROUP BY value    
ORDER BY COUNT(*) DESC;

(28 rows affected)

Total execution time: 00:00:00.014

Language,Count
English,244
Hindi,44
Japanese,26
French,15
Korean,9
Spanish,8
German,8
Russian,8
Turkish,7
Swedish,6


**Genre analysis:** Most movies fell under multiple genres, but I was surprised to see that despite this, there were multiple combinations of genres that topped the list. I noticed, however, that te top categories appeared to contain drama as a genre.

In [100]:
--Breakdown by genre #1 
SELECT genre, COUNT(Title) as 'count'
FROM MoviesDB.dbo.IMDBtop400
GROUP BY genre
ORDER BY COUNT(Title) DESC;

(130 rows affected)

Total execution time: 00:00:00.012

genre,count
Drama,37
"Crime,Drama",16
"Biography,Drama,History",15
"Crime,Drama,Thriller",13
"Drama,Romance",13
"Drama,War",11
"Crime,Drama,Mystery",10
"Comedy,Drama,Romance",10
"Animation,Adventure,Comedy",10
"Biography,Drama",10


Still, I thought it would be useful to double count the movies with multiple genres. An overwhelming majority of the movies were classified as "drama," with crime, adventure, comedy, and action also being popular.

In [102]:
-- Breakdown by genre #2 
SELECT value as 'Genre', COUNT(*) AS 'Count'
FROM MoviesDB.dbo.IMDBtop400  
    CROSS APPLY STRING_SPLIT(genre, ',')  
GROUP BY value    
ORDER BY COUNT(*) DESC;

(21 rows affected)

Total execution time: 00:00:00.010

Genre,Count
Drama,302
Crime,81
Adventure,80
Comedy,76
Action,75
Thriller,58
Biography,46
Romance,45
Mystery,43
Animation,34


**Popular directors:** Undoubtedly, there are some very well-known names in the list of 400 movies. I decided to query the most popular directors by splitting up the "cast" column, which contained both the actors and directors of the movie (probably due to the nature of data scraping). Surprisingly, 67 directors showed up at least twice, and one, Akira Kurosawa, even beat out the big names I am familiar with, such as Stanley Kubrick, Alfred Hitchcock and Martin Scorsese, with ten movies!

In [103]:
--Popular directors 
SELECT Director, COUNT(Title) AS 'Count'
FROM
-- Add a directors column 
(SELECT *,
    CASE WHEN Cast LIKE '%Directors%' 
    THEN SUBSTRING(Cast, CHARINDEX('Directors: ', Cast) + 11, CHARINDEX('Stars: ', Cast) - 13)
    ELSE SUBSTRING(Cast, CHARINDEX('Director: ', Cast) + 10, CHARINDEX('Stars: ', Cast) - 13)
    END AS 'Director' 
FROM MoviesDB.dbo.IMDBtop400) AS T
GROUP BY Director
HAVING COUNT(Title) > 1
ORDER BY Count DESC;


(67 rows affected)

Total execution time: 00:00:00.025

Director,Count
Akira Kurosawa,10
Christopher Nolan,7
Martin Scorsese,7
Stanley Kubrick,7
Steven Spielberg,6
Charles Chaplin,6
Quentin Tarantino,6
Alfred Hitchcock,6
Billy Wilder,6
Ingmar Bergman,6


**Popular actors:** Next, I wanted to see what actors were popular. The most popular American actor was Robert De Niro, while India's Aamir Khan and Japan's Toshiro Mifune followed closely. I thought the gender gap was interesting; the first female name on the list is Scarlett Johansson, who appeared in four movies (and only plays a main character in one).

In [109]:
--Popular actors 
 SELECT value as 'Star', COUNT(*) AS 'Count'
FROM 
-- Creates new column with just the actors 
(SELECT *, 
    SUBSTRING(Cast, CHARINDEX('Stars: ', Cast) + 7, len(Cast)) AS 'Stars' 
FROM MoviesDB.dbo.IMDBtop400) AS T  
    CROSS APPLY STRING_SPLIT(Stars, ',')  
GROUP BY value 
HAVING COUNT(*) > 1    
ORDER BY COUNT(*) DESC;

(218 rows affected)

Total execution time: 00:00:00.030

Star,Count
Robert De Niro,9
Leonardo DiCaprio,8
Aamir Khan,8
Toshiro Mifune,7
Harrison Ford,7
James Stewart,6
Charles Chaplin,6
Brad Pitt,6
Tom Hanks,6
Matt Damon,6


**Breakdown by release year:** I also wanted to the see the distribution of the movies by year. I noticed that, unsurprisingly, newer movies are overrepresented, as they tend to have a larger modern day following. As a result, I decided to increment release years by 10 until 1990, after which I decided to increment the years by 5. Unsurprisingly, the most recent decade has the most movies in the list.

In [272]:
-- Breakdown by year of release 
(SELECT '2015 - 2020' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 2014)
UNION ALL 
(SELECT '2010 - 2014' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 2009 AND Year < 2015) 
UNION ALL
(SELECT '2005 - 2009' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 2004 AND Year < 2010)
UNION ALL
(SELECT '2000 - 2004' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 1999 AND Year < 2005)
UNION ALL
(SELECT '1995 - 1999' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 1994 AND Year < 2000)
UNION ALL
(SELECT '1990 - 1994' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 1989 AND Year < 1995)
UNION ALL
SELECT '1980 - 1989' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 1979 AND Year < 1990
UNION ALL
SELECT '1970 - 1979' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 1969 AND Year < 1980
UNION ALL
SELECT '1960 - 1969' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 1959 AND Year < 1970
UNION ALL
SELECT '1950 - 1959' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 1949 AND Year < 1960
UNION ALL
SELECT '1940 - 1949' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year > 1939 AND Year < 1950
UNION ALL
SELECT 'Before 1940' AS 'Year', COUNT(*) AS 'COUNT' FROM
(SELECT Title, CAST(SUBSTRING(Title, LEN(Title) - 4, 4) AS int) AS 'Year'
FROM MoviesDB.dbo.IMDBtop400) AS T
WHERE Year < 1940; 

(12 rows affected)

Total execution time: 00:00:00.030

Year,COUNT
2015 - 2020,52
2010 - 2014,53
2005 - 2009,41
2000 - 2004,49
1995 - 1999,42
1990 - 1994,23
1980 - 1989,31
1970 - 1979,25
1960 - 1969,27
1950 - 1959,28
